In [ ]:
# https://www.mdpi.com/2076-3417/12/19/10077

In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd
from rasterio.sample import sample_gen
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,ConfusionMatrixDisplay,\
                            precision_score,recall_score,f1_score,roc_auc_score, roc_curve

# import matplotlib as plt
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)
import joblib
import os
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



### Load sample points and sar data


In [2]:
# Load SAR GeoTIFF
sar_image_path = "output/sar_image/eaton_sar_indices.tif"

# Load the CSV data
# training_data_path = 'sample_1_training_data.csv'
# training_data_path="sample_1_training_data_texture.csv"
training_data_path="output/sample/training_data_eaton_indices.csv"

training_data=pd.read_csv(training_data_path)
# feature_column_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']
# "dTRAD","dNBR",
# feature_column_names=["vh_post","vv_post","vh_pre","vv_pre"]
feature_column_names=["RBD_VV", "RBD_VH", "RBR_VV","dod_RBR_VV", "RBR_VH","dod_RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]


# feature_column_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV","dod_RBR_VV", "RBR_VH","dod_RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'] # "ΔRVI",'Δvv_vh_ratio',"ΔRDFI" "RBD_VV",

# feature_column_names=["RBD_VV", "RBD_VH", "RBR_VV", "dod_RBR_VV","RBR_VH","dod_RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'] #,"ΔRVI",'Δvv_vh_ratio',"ΔRDFI","dTRAD"

# feature_column_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI","Δvv_vh_ratio","ΔRDFI"] #
# feature_column_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']
# feature_column_names=["gamm_vh_pre","gamm_vv_pre","gamm_vh_post","gamm_vv_post","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI"]
# feature_column_names=["gamm_vh_pre","gamm_vv_pre","gamm_vh_post","gamm_vv_post","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']

class_column_name='class'
extended_file_name="_reproject"


output_model_dir="output/model/"


In [3]:
dataset = rasterio.open(sar_image_path)
# Get the number of bands
num_bands = dataset.count
print(f"Number of bands: {num_bands}")

# To get the names of the bands (if available)
# for i in range(1, num_bands + 1):
#     band_name = dataset.tags(i)  # Retrieve tags for the band
#     print(f"Band {i} name: {band_name}")

Number of bands: 10


In [4]:
# Separate features (SAR bands) and labels (e.g., class column)
X = training_data[feature_column_names]  # Features (SAR band values)
Y = training_data[class_column_name]  # Labels (add 'Class' column in your CSV if available)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)


In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import train_test_split
# import numpy as np

# # Train-test split (assuming X_train and y_train are already defined)
# # X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# # Initialize the model
# gb_model = GradientBoostingClassifier(
#     n_estimators=1000,        # Allow a large number of trees initially
#     learning_rate=0.05,
#     max_depth=3,
#     min_samples_split=10,
#     min_samples_leaf=5,
#     subsample=0.8,
#     random_state=42
# )

# # Initialize variables for early stopping
# best_score = -np.inf
# patience = 50  # Number of rounds to wait for improvement
# no_improvement_count = 0

# # Loop over the number of estimators
# for i in range(1, gb_model.n_estimators + 1):
#     # Fit the model up to the i-th iteration
#     gb_model.n_estimators = i
#     print((gb_model.n_estimators))
#     gb_model.fit(X_train, y_train)
    
#     # Calculate the validation score (e.g., accuracy)
#     val_score = gb_model.score(X_test, y_test)
#     print(val_score)
#     # Check if the validation score has improved
#     if val_score > best_score:
#         best_score = val_score
#         no_improvement_count = 0  # Reset the counter
#     else:
#         no_improvement_count += 1
    
#     # If no improvement for 'patience' rounds, stop training
#     if no_improvement_count >= patience:
#         print(f"Early stopping at iteration {i}")
#         break

# # Final model after early stopping
# print(f"Best validation score: {best_score}")


1
0.689119170984456
2
0.689119170984456
3
0.689119170984456
4
0.689119170984456
5
0.689119170984456
6
0.689119170984456
7
0.6981865284974094
8
0.7098445595854922
9
0.7150259067357513
10
0.7202072538860104
11
0.7215025906735751
12
0.7240932642487047
13
0.7253886010362695
14
0.7292746113989638
15
0.7331606217616581
16
0.7363989637305699
17
0.7338082901554405
18
0.7344559585492227
19
0.7383419689119171
20
0.7402849740932642
21
0.7422279792746114
22
0.7435233160621761
23
0.7448186528497409
24
0.7441709844559585
25
0.7454663212435233
26
0.7454663212435233
27
0.7454663212435233
28
0.7474093264248705
29
0.7474093264248705
30
0.7487046632124352
31
0.7487046632124352
32
0.7487046632124352
33
0.7512953367875648
34
0.7525906735751295
35
0.7519430051813472
36
0.7512953367875648
37
0.7525906735751295
38
0.7525906735751295
39
0.7525906735751295
40
0.7538860103626943
41
0.7512953367875648
42
0.7519430051813472
43
0.7525906735751295
44
0.7532383419689119
45
0.7532383419689119
46
0.7525906735751295
47


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# import numpy as np

# # Assuming X_train_full and y_train_full are already defined
# # Train-test split
# # X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# # Initialize the Random Forest model with a large number of trees
# rf_model = RandomForestClassifier(
#     n_estimators=1000,  # Allow a large number of trees initially
#     max_depth=10,
#     min_samples_split=2,
#     min_samples_leaf=1,
#     random_state=42
# )

# # Initialize variables for early stopping
# best_score = -np.inf
# patience = 50  # Number of rounds to wait for improvement
# no_improvement_count = 0

# # Loop over the number of estimators (trees)
# for i in range(1, rf_model.n_estimators + 1):
#     # Incrementally fit the Random Forest model with the i-th number of trees
#     rf_model.set_params(n_estimators=i)
#     rf_model.fit(X_train, y_train)
    
#     # Calculate the validation score (e.g., accuracy)
#     val_score = rf_model.score(X_test, y_test)
#     print(f"Iteration {i}, Validation Score: {val_score}")
    
#     # Check if the validation score has improved
#     if val_score > best_score:
#         best_score = val_score
#         no_improvement_count = 0  # Reset the counter
#     else:
#         no_improvement_count += 1
    
#     # If no improvement for 'patience' rounds, stop training
#     if no_improvement_count >= patience:
#         print(f"Early stopping at iteration {i}")
#         break

# # Final model after early stopping
# print(f"Best validation score: {best_score}")


Iteration 1, Validation Score: 0.7137305699481865
Iteration 2, Validation Score: 0.707901554404145
Iteration 3, Validation Score: 0.727979274611399
Iteration 4, Validation Score: 0.7344559585492227
Iteration 5, Validation Score: 0.7331606217616581
Iteration 6, Validation Score: 0.7305699481865285
Iteration 7, Validation Score: 0.7383419689119171
Iteration 8, Validation Score: 0.7396373056994818
Iteration 9, Validation Score: 0.7428756476683938
Iteration 10, Validation Score: 0.7422279792746114
Iteration 11, Validation Score: 0.7409326424870466
Iteration 12, Validation Score: 0.7402849740932642
Iteration 13, Validation Score: 0.7402849740932642
Iteration 14, Validation Score: 0.741580310880829
Iteration 15, Validation Score: 0.7428756476683938
Iteration 16, Validation Score: 0.7441709844559585
Iteration 17, Validation Score: 0.7448186528497409
Iteration 18, Validation Score: 0.7441709844559585
Iteration 19, Validation Score: 0.7474093264248705
Iteration 20, Validation Score: 0.746761658

In [ ]:
# from sklearn.ensemble import StackingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Define base learners (models to be stacked)
# base_learners = [
#     ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
#     ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
#     ('svm', SVC(probability=True, random_state=42))
# ]

# # Meta-model (Logistic Regression)
# meta_model = LogisticRegression()

# # Create the Stacking Classifier
# stacking_model = StackingClassifier(estimators=base_learners, final_estimator=meta_model)

# # Train the Stacking model
# stacking_model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred_stack = stacking_model.predict(X_test)

# # Evaluate the Stacking model's performance
# accuracy_stack = accuracy_score(y_test, y_pred_stack)
# print(f"Accuracy of Stacking model: {accuracy_stack:.2f}")


Accuracy of Stacking model: 0.75


In [ ]:
#indices_tex
# RF_Model_1=RandomForestClassifier(n_estimators= 150, min_samples_split= 8, max_features= 9, max_depth= 10)

#gamma_indices
# RF_Model_1=RandomForestClassifier(n_estimators= 100, min_samples_split= 8, max_features= 20, max_depth= 8)

#gamma
# RF_Model_0=RandomForestClassifier(n_estimators= 150, min_samples_split= 15, max_features= 20, max_depth= 12)
# RF_Model_1=RandomForestClassifier(n_estimators= 200, min_samples_split= 8, max_features= 20, max_depth= 8)
# {'n_estimators': 150, 'min_samples_split': 20, 'max_features': 4, 'max_depth': 15}
#indices
# RF_Model_0=RandomForestClassifier(n_estimators= 150, min_samples_split= 4, max_features= 20, max_depth= 10)
# RF_Model_1=RandomForestClassifier(n_estimators= 150, min_samples_split= 8, max_features= 7, max_depth= 8)


#eaton indices
# RF_Model_0=RandomForestClassifier(n_estimators= 200,random_state=42)
# max_depth= 8
# {'n_estimators': 100, 'min_samples_split': 40, 'max_features': 3, 'max_depth': 8}
RF_Model_0=RandomForestClassifier(n_estimators= 100, min_samples_split= 40, max_features= 3,max_depth= 8,bootstrap=True)
# RF_Model_1=RandomForestClassifier(n_estimators= 100, min_samples_split= 4, max_features= 8, max_depth= 5)
# RF_Model_0 = RandomForestClassifier(
#     n_estimators=100,               # Number of trees in the forest
#     max_depth=None,                  # Trees can grow deep to capture complex patterns
#     min_samples_split=2,             # Minimum samples required to split an internal node
#     min_samples_leaf=1,              # Minimum samples required to be at a leaf node
#     max_features='sqrt',             # Use square root of the total features for each split
#     bootstrap=True,                  # Use bootstrap sampling (default)
#     n_jobs=-1,                       # Use all available processors
#     random_state=42                  # Ensures reproducibility
# )

# SVM_Model = svm.SVC()
SVM_Model= LinearSVC(random_state=42, tol=1e-3)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()

models={
    "RandomForest0": RF_Model_0,
    "SVM_Model": SVM_Model
    # "RandomForest1": RF_Model_1,
    # "RandomForest2": RF_Model_2,
    # "RandomForest3": RF_Model_3,
    # "RandomForest4": RF_Model_4,
    # "RandomForest5": RF_Model_5,

    # "Decision Tree":DecisionTreeClassifier(),
    # "Logistic Regression":LogisticRegression()
    # "XGBoost": XGB_Model
}


for i in range(len(list(models))):
    model_name=list(models.keys())[i]
    print(f"***************{model_name}*********************")
    model=list(models.values())[i]

    # scaler = StandardScaler()
    # pixels_scaled = scaler.fit_transform(pixels)  # Apply scaling to the pixels
    # X_train=scaler.fit_transform(X_train)
    model.fit(X_train, y_train)
    # print("score",model.score(X_test,y_test))

    #Make a prediction
    y_train_pred=model.predict(X_train)
    # y_test_pred=model.predict(scaler.transform(X_test))
    y_test_pred=model.predict(X_test)


    #Training set performance
    model_train_accuracy=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred,average='weighted')
    model_train_precision=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_roc_auc_score=roc_auc_score(y_train,y_train_pred)

    #Test set performance
    model_test_accuracy=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred,average='weighted')
    model_test_precision=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_roc_auc_score=roc_auc_score(y_test,y_test_pred)

    # print(model)
    print('Model performance for Training set')
    print(f"- Accuracy: {model_train_accuracy}")
    print(f"- F1 Score: {model_train_f1}")
    print(f"- Precision Score: {model_train_precision}")
    print(f"- Recall Score: {model_train_recall}")
    print(f"- Roc Auc Score: {model_train_roc_auc_score}")

    print("--------------")
    print('Model performance for Test set')
    print(f"- Accuracy: {model_test_accuracy}")
    print(f"- F1 Score: {model_test_f1}")
    print(f"- Precision Score: {model_test_precision}")
    print(f"- Recall Score: {model_test_recall}")
    print(f"- Roc Auc Score: {model_test_roc_auc_score}")
    print("----------------------------------------")

    
    # Assuming you have a trained model called `model`
    joblib.dump(model, f'{output_model_dir}{model_name}.pkl')

    print("Model saved successfully!")

    # predict(model, sar_image_path,model_name,scaler)
    # predict(model, sar_image_path,model_name)

    # predict(model, "output/sar_image/palisades_sar_indices.tif",model_name)



# Initialize the Random Forest classifier
# rf_classifier = RandomForestClassifier(n_estimators= 100)

# # Train the model
# rf_classifier.fit(X_train, y_train)

# # Evaluate the model
# accuracy = rf_classifier.score(X_test, y_test)
# print(f"Random Forest Accuracy: {accuracy * 100:.2f}%")


# Model performance for Training set
# - Accuracy: 0.9043069948186528
# - F1 Score: 0.9032436217125988
# - Precision Score: 0.8647854203409759
# - Recall Score: 0.8029475982532751
# - Roc Auc Score: 0.8750005026257168
# --------------
# Model performance for Test set
# - Accuracy: 0.8853626943005182
# - F1 Score: 0.8834883526398328
# - Precision Score: 0.8531468531468531
# - Recall Score: 0.7625
# - Roc Auc Score: 0.8516447368421053

In [ ]:
# Create a series containing feature importances from the model and feature names from the training data

feature_importances = pd.Series(RF_Model_0.feature_importances_, index=X_train.columns).sort_values(ascending=False)
# Plot a simple bar chart
feature_importances.plot.bar()

In [ ]:
import seaborn as sns
corr_mat_export_path="output/correlation/cor_mat.csv"
# Compute Pearson correlation matrix
training_data_1 = training_data.drop(columns=['id','class','geometry'])
correlation_matrix = training_data_1.corr(method='pearson').round(2)
print(correlation_matrix)
correlation_matrix = correlation_matrix.applymap(lambda x: x if x >= 0.7 else 0)
correlation_matrix.to_csv(corr_mat_export_path, index=True)

# corr_matrix = df.corr()

# Create heatmap
plt.figure(figsize=(8,6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()


## prediction for entire image

In [ ]:
def predict(model, image_path,model_name ):
    dataset = rasterio.open(image_path)
    print(dataset.count)
    # # Load the SAR GeoTIFF
    # image_path = "../SAR_Data_processing/11_export/palisades_indices_subset.tif"
    # dataset = rasterio.open(image_path)

    # Read the image bands into an array (assuming it's a multi-band raster)
    sar_bands = np.stack([dataset.read(i+1) for i in range(dataset.count)], axis=-1)

    # Reshape the SAR data to a 2D array (num_pixels, num_bands)
    height, width, num_bands = sar_bands.shape
    pixels = sar_bands.reshape(-1, num_bands)  # Each row is a pixel
    print(num_bands,"num_bands")

    # Preprocess (scale) the pixel values if needed (based on training data preprocessing)
    # scaler = StandardScaler()
    # pixels_scaled = scaler.fit_transform(pixels)  # Apply scaling to the pixels

    # Predict for each pixel
    predictions = model.predict(pixels)

    from scipy.ndimage import uniform_filter
    # Assuming predictions are a 2D array (for an image or spatial data)
    predictions = uniform_filter(predictions, size=7)  # size is the window size

    # predictions = model.predict(scaler.transform(pixels))


    # Reshape predictions to match the image dimensions
    predicted_image = predictions.reshape(height, width)

    # Save the predicted classes to a new file
    output_path = f"output/prediction/eaton_{model_name}_{extended_file_name}.tif"
    meta = dataset.meta
    meta.update(dtype=rasterio.uint8, count=1)  # Assuming class labels are integers, uint8 works for this

    with rasterio.open(output_path, 'w', **meta) as dest:
        dest.write(predicted_image.astype(rasterio.uint8), 1)  # Write to the first band

    print(f"Predictions saved to {output_path}")

    pred_dataset = rasterio.open(output_path)

    # Read the first band of the image (you can adjust for multi-band images)
    pred_band_1 = pred_dataset.read(1)

    # Plot the image
    plt.figure(figsize=(10, 10))
    plt.imshow(pred_band_1, cmap='gray')  # You can adjust the colormap if needed
    plt.colorbar()
    plt.title("SAR Burnt Area Prediction Map")
    plt.show()


In [ ]:
ml_models=os.listdir(output_model_dir)
for model_name in ml_models:
    output_model_path = os.path.join(output_model_dir, model_name)
    # Load the saved model
    loaded_model = joblib.load(output_model_path)

    # Use the loaded model to make predictions
    # predictions = loaded_model.predict(X_test)
    predict(loaded_model, sar_image_path,model_name.replace(".pkl",""))

## plot AUC curve

In [ ]:
## plot ROC AUC Curve
plt.figure()
auc_models = [{
    'label': 'Random Forest Classifier',
    'model': RF_Model,
    'auc':0.73
}]

for algo in auc_models:
    model=algo['model']
    model.fit(X_train, y_train)
    fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
    plt.plot(fpr,tpr,label="Random Forest Classifier ROC (area= 0.71)")

    plt.plot([0,1],[0,1],'r--')
    plt.xlim([0.0,1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('1-Specificity (False Positive Rate)')
    plt.ylabel('Sensitivity (True Positive Rate)')
    plt.title('Receiver operating characteristics')

    plt.legend(loc="lower right")

    plt.savefig("auc.png")
    plt.show()

    # Open the TIFF file using rasterio
    # output_path = "palisades_predictions.tif"  # Replace with your TIFF file path
    
